### Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import numpy as np

###  dataset path

In [2]:
root_dir = "../data"  # Change this to your dataset folder

# Load dataset (Train & Validation)
batch_size = 32
img_size = (224, 224)

train_dataset = image_dataset_from_directory(
    root_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_dataset = image_dataset_from_directory(
    root_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


Found 1835 files belonging to 30 classes.
Using 1468 files for training.
Found 1835 files belonging to 30 classes.
Using 367 files for validation.


### Class names

In [3]:
class_names = train_dataset.class_names
num_classes = len(class_names)
print(f"Classes: {class_names}, Total Classes: {num_classes}")

# Normalize and preprocess images
def preprocess(image, label):
    image = preprocess_input(image)  # Apply EfficientNet preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess).cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


Classes: ['Alpinia Galanga (Rasna)', 'Amaranthus Viridis (Arive-Dantu)', 'Artocarpus Heterophyllus (Jackfruit)', 'Azadirachta Indica (Neem)', 'Basella Alba (Basale)', 'Brassica Juncea (Indian Mustard)', 'Carissa Carandas (Karanda)', 'Citrus Limon (Lemon)', 'Ficus Auriculata (Roxburgh fig)', 'Ficus Religiosa (Peepal Tree)', 'Hibiscus Rosa-sinensis', 'Jasminum (Jasmine)', 'Mangifera Indica (Mango)', 'Mentha (Mint)', 'Moringa Oleifera (Drumstick)', 'Muntingia Calabura (Jamaica Cherry-Gasagase)', 'Murraya Koenigii (Curry)', 'Nerium Oleander (Oleander)', 'Nyctanthes Arbor-tristis (Parijata)', 'Ocimum Tenuiflorum (Tulsi)', 'Piper Betle (Betel)', 'Plectranthus Amboinicus (Mexican Mint)', 'Pongamia Pinnata (Indian Beech)', 'Psidium Guajava (Guava)', 'Punica Granatum (Pomegranate)', 'Santalum Album (Sandalwood)', 'Syzygium Cumini (Jamun)', 'Syzygium Jambos (Rose Apple)', 'Tabernaemontana Divaricata (Crape Jasmine)', 'Trigonella Foenum-graecum (Fenugreek)'], Total Classes: 30


### Load pre-trained EfficientNetB0

In [4]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dense(num_classes, activation="softmax")(x)  # Multi-class classification

# Create final model
model = Model(inputs=base_model.input, outputs=x)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['normalization[0][0]']      

 block2a_project_conv (Conv2D)  (None, 56, 56, 24)   2304        ['block2a_se_excite[0][0]']      
                                                                                                  
 block2a_project_bn (BatchNorma  (None, 56, 56, 24)  96          ['block2a_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block2b_expand_conv (Conv2D)   (None, 56, 56, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 56, 56, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_e

 block3b_expand_bn (BatchNormal  (None, 28, 28, 240)  960        ['block3b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block3b_expand_activation (Act  (None, 28, 28, 240)  0          ['block3b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 28, 28, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 28, 28, 240)  960        ['block3b_dwconv[0][0]']         
 )        

 D)                                                              ']                               
                                                                                                  
 block4b_bn (BatchNormalization  (None, 14, 14, 480)  1920       ['block4b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block4b_activation (Activation  (None, 14, 14, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
          

 block5a_activation (Activation  (None, 14, 14, 480)  0          ['block5a_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block5a_se_squeeze (GlobalAver  (None, 480)         0           ['block5a_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_s

 block5c_se_expand (Conv2D)     (None, 1, 1, 672)    19488       ['block5c_se_reduce[0][0]']      
                                                                                                  
 block5c_se_excite (Multiply)   (None, 14, 14, 672)  0           ['block5c_activation[0][0]',     
                                                                  'block5c_se_expand[0][0]']      
                                                                                                  
 block5c_project_conv (Conv2D)  (None, 14, 14, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 14, 14, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_d

 block6b_project_bn (BatchNorma  (None, 7, 7, 192)   768         ['block6b_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block6b_drop (Dropout)         (None, 7, 7, 192)    0           ['block6b_project_bn[0][0]']     
                                                                                                  
 block6b_add (Add)              (None, 7, 7, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_e

 block6d_add (Add)              (None, 7, 7, 192)    0           ['block6d_drop[0][0]',           
                                                                  'block6c_add[0][0]']            
                                                                                                  
 block7a_expand_conv (Conv2D)   (None, 7, 7, 1152)   221184      ['block6d_add[0][0]']            
                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 7, 7, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 7, 7, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
          

### Train the model

In [5]:
epochs = 5  # Adjust as needed
history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)


Epoch 1/5
46/46 [==============================] - 78s 2s/step - loss: 1.7389 - accuracy: 0.5988 - val_loss: 0.6666 - val_accuracy: 0.8992
Epoch 2/5
46/46 [==============================] - 69s 2s/step - loss: 0.4074 - accuracy: 0.9278 - val_loss: 0.2584 - val_accuracy: 0.9673
Epoch 3/5
46/46 [==============================] - 67s 1s/step - loss: 0.1982 - accuracy: 0.9721 - val_loss: 0.1698 - val_accuracy: 0.9728
Epoch 4/5
46/46 [==============================] - 66s 1s/step - loss: 0.1175 - accuracy: 0.9877 - val_loss: 0.1157 - val_accuracy: 0.9864
Epoch 5/5
46/46 [==============================] - 67s 1s/step - loss: 0.0826 - accuracy: 0.9925 - val_loss: 0.0890 - val_accuracy: 0.9837


### Load test dataset 

In [6]:
test_dataset = image_dataset_from_directory(
    root_dir,
    image_size=img_size,
    batch_size=batch_size
)

# Preprocess test dataset
test_dataset = test_dataset.map(preprocess).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Evaluate model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")


Found 1835 files belonging to 30 classes.
58/58 [==============================] - 72s 1s/step - loss: 0.0541 - accuracy: 0.9962
Test Accuracy: 0.9962


### Get true labels and predictions and classification report

In [7]:
y_true, y_pred = [], []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(labels.numpy())
    
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))


1/1 [==============================] - 2s 2s/step
                                              precision    recall  f1-score   support

                     Alpinia Galanga (Rasna)     1.0000    1.0000    1.0000        50
            Amaranthus Viridis (Arive-Dantu)     1.0000    1.0000    1.0000       122
        Artocarpus Heterophyllus (Jackfruit)     0.9825    1.0000    0.9912        56
                   Azadirachta Indica (Neem)     1.0000    1.0000    1.0000        60
                       Basella Alba (Basale)     1.0000    1.0000    1.0000       103
            Brassica Juncea (Indian Mustard)     1.0000    1.0000    1.0000        34
                  Carissa Carandas (Karanda)     1.0000    1.0000    1.0000        74
                        Citrus Limon (Lemon)     1.0000    1.0000    1.0000        57
             Ficus Auriculata (Roxburgh fig)     1.0000    0.9800    0.9899        50
               Ficus Religiosa (Peepal Tree)     1.0000    1.0000    1.0000        63
   